In [1]:
import os
import sys
import gzip
import code
import urllib
import importlib
import tensorflow.python.platform
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm
from PIL import Image
from datetime import datetime

C:\Users\jbenh\anaconda3\envs\tf-gpu2\lib\site-packages\ipykernel\parentpoller.py:113: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


In [5]:
COLAB = False
BRANCH = 'main'

In [6]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/Shareddrives/ML_Road_Segmentation/CS-433-project-ben/project_road_segmentation'
    os.chdir(drive_path)
    from helpers.colab import mount_and_pull
    BRANCH_NAME = BRANCH
    mount_and_pull(BRANCH_NAME, drive, os)

In [7]:
%load_ext autoreload
%autoreload 2

from helpers.image_processing import *
from helpers.file_manipulation import *
from helpers.constants import *
from helpers.prediction_checking import *
from helpers.machine_learning import *
from helpers.loss_functions import *
from helpers.submission import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
RESTORE_MODEL = True  # If True, restore existing model instead of training a new one
SAVE_MODEL = False
GENERATE_PREDICTION = True  # If True, will generate a CSV to submit on AICrowd

MODEL_NAME = 'unet_2'  # For now, cnn, unet-1, unet-2
PATCH_PREDICTION = True # If using the CNN, set to True
SAVE_DIR = MODELS_SAVE_DIR + MODEL_NAME + '/'

NUM_EPOCHS = 250

In [6]:
mod = importlib.import_module('models.' + MODEL_NAME)
model_function = getattr(mod, MODEL_NAME)

In [7]:
FEATURE_AUGMENTATION = 2
X_train, y_train, X_test, y_test = get_train_test(data_augmentation=True,transformations=['mix','flip', 'rotation'])

Loading data/generated/rotation/groundtruth/: 100%|█████████████████████████████████| 100/100 [00:00<00:00, 259.06it/s]


Training features shape :  (400, 400, 400, 3)
Training labels shape :  (400, 400, 400)


In [8]:
if PATCH_PREDICTION:
    from helpers.patch_labeling import *
    old_y_train = y_train
    old_y_test = y_test
    y_train = label_to_patch_prediction(y_train)
    y_test = label_to_patch_prediction(y_test)
    print(y_train.shape)
    print(y_test.shape)
    visualize_patches(old_y_train, y_train, number=3, resize=False)

In [10]:
metrics=[
         tf.keras.metrics.Precision(),
         tf.keras.metrics.Recall(),
         dice_coef
        ]
callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5)
            ]
if RESTORE_MODEL:
    model = tf.keras.models.load_model(SAVE_DIR, custom_objects={'focal_tversky_loss': focal_tversky_loss, 'dice_coef': dice_coef})
else:  
    model = model_function(feature_augmentation=FEATURE_AUGMENTATION+1)
    model.compile(optimizer='adam',
                loss=focal_tversky_loss,
                metrics=metrics)
    history = model.fit(X_train, y_train, 
                        epochs = NUM_EPOCHS,
                        validation_split=0.2,
                        callbacks=callbacks)

In [ ]:
if SAVE_MODEL:
    now = datetime.now().strftime("%m-%d-%Y_%H-%M-%S") + '/'
    model.save(SAVE_DIR + new)

In [ ]:
train_predictions = model.predict(X_train).squeeze()
test_predictions = model.predict(X_test).squeeze()

In [ ]:
metrics = compute_entire_images_metrics(y_test, test_predictions)

In [ ]:
metrics

In [ ]:
if PATCH_PREDICTION:
    train_predictions = resize_images(train_predictions, TRAINING_IMG_SIZE)
    test_predictions = resize_images(test_predictions, TRAINING_IMG_SIZE)

In [ ]:
visualize_random_predictions(X_train, y_train, train_predictions, size=10)

In [ ]:
visualize_random_predictions(X_test, y_test, test_predictions, size=10)

In [32]:
with tf.device('cpu'):
    predict_submissions(model, patches_predicted=PATCH_PREDICTION)

Loading data/test_set_images/: 100%|███████████████████████████████████████████████████| 50/50 [00:00<00:00, 62.40it/s]


Predicting test images...
Creating submission csv at location submissions/submission_12-15-2020_11-50-40.csv
Writing predictions in folder data/predictions/


In [ ]:
if COLAB:
    from helpers.colab import download_model
    from google.colab import files
    download_model(MODEL_NAME, SAVE_DIR, files)